<a href="https://www.kaggle.com/code/akouaorsot/musical-analytics-spotify-2010-2022?scriptVersionId=143500575" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

With the rise of musical softwares such as Soundcloud, Deezer, and Spotify, there has a expansion in the access of an extensive music library while breaking down barriers for upcoming artists to share their music with others. In so doing, there is now more traceability on the songs/albumns catching the most attention and popularity. With this newly found pool of data points, music and entertainment industry would be greatly served in leveraging data analytics to tailor the musical offering onto this market. 

It will be our modest attempt to conduct different analyses onto this 2010-2022 Spotify dataset to enhance the quality of the dataset, explore a variety of questions, run some tests, and draw some insights.

The work of [James Chryssanthacopoulos](https://towardsdatascience.com/advanced-music-analytics-using-machine-learning-f344e4795bbc) was also a great inpsiration

# 1. Set-up

In [ ]:
## Importing Libraries
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re           # Regex Text manipulation
import itertools    # For iterators and efficent looping
from collections import Counter


## Data Visualization
import matplotlib.pyplot as plt     # data visualization
import seaborn as sns               # Other visualization package
sns.set_style("darkgrid")


In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
paths = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename)) 
#         print(os.path.join(dirname, filename))

In [ ]:
# Loading the data
df = pd.read_csv(paths[0])
df.head()

In [ ]:
df.info()

#### As noted above, there are 23 columns of which 13 represents audio features of any given song. To have a deeper understanding of the underlying information captured, will we rely on the following sources to shed light on the meaning of each feature. The following source was of great help: [SPOTONTRACK - What do the audio features mean?](https://help.spotontrack.com/article/what-do-the-audio-features-mean)

* **Danceability**: describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity.

* **Energy**: representing a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
 
* **Key**: Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.

* **Mode**: indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.

* **Loudness**: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.

* **Speechiness**: detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

* **Acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic; thus indicating the likelihood of electrical amplification.

* **Instrumentalness**: whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content.

* **Liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. 

* **Valence**: describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

* **Tempo**: estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

* **Time Signature**: indicates how many counts are in each measure and which type of note will receive one count.

# 2. Data Cleaning

In [ ]:
# Since the following columns do not hold 
# as much pertinent information, we will drop them

df.drop(columns=['playlist_url', 'track_id', 'artist_id'],
       inplace=True)

In [ ]:
# Shape of data: No. of rows and columns
no_rows, no_cols = df.shape

# Check for missing values
null_values = df.isnull().sum().sum()

# No. of duplicates values
duplicates_n = df[df.duplicated()].shape[0]


# Diagnostics results
data_check = """
- The data has {0} rows and {1} columns
- There are {2} null/na values and {3} duplicated rows. 
""" 

print(data_check.format(no_rows, no_cols,
                        null_values, duplicates_n))

In [ ]:
# A closer look shows row 447 has na/null values
# mostly in the audio features columns.
df[df.isnull().any(axis=1)]

# Comment 
# df[(df['year'] == 2004) & (df['artist_genres'].str.find('pop') > -1)]

# Since it's only 1 observation, we will drop it
df.drop([447], inplace=True)

# 3. Exploratory Data Analysis

In [ ]:
# Descriptive Statistics - Quick overview
df.describe()

In [ ]:
## Multiplot of distribution
# Proportions by seat type
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(5, 3, hspace=0.5, wspace=0.5)
axs = gs.subplots()
fig.suptitle("Distribution of All Audio Features")
axs[0,0].hist(df['track_popularity'])
# ax1.pie(labels=seat_type, x=seat_counts, autopct='%.0f%%')
axs[0,0].set_title('track_popularity')
# ax2.pie(labels=types, x=type_counts , autopct='%.0f%%')
# ax2.set_title("${Traveller}$ ${Type}$")

# fig.tight_layout(pad=.75)
#plt.title('Breakdown of Seat Type & Traveller Type at BA Flights 2016-2023')
plt.show()

# Potential Pathways
* **Music analysis**: Explore patterns in audio features to understand trends and preferences in popular songs.
* **Artist impact**: Analyze how artist involvement and attributes relate to a song's success.
* **Temporal trends**: Identify any shifts in music attributes and preferences over time.
* Segmentation by genres -> Clustering 
* Corrrelaiton between artist popularity and track
* Audio features across time, most notable increase/decrease
* If artist has substring of certain genres, create a K-means (others) clustering model for track/artist popularity and other audio feature

In [ ]:
df['artist_genres_clean'] = df['artist_genres'].apply(lambda st: re.sub("\['|'\]|'", "", st).split(", "))

genres = [genre for genre in df['artist_genres_clean']]
all_genres = list(itertools.chain.from_iterable(genres))
# list(filter(lambda k: 'hip hop' in k, all_genres))
# Counter(all_genres)

# Machine Learning

In [ ]:
## Develop a model and use the 2023 as test data